<a href="https://colab.research.google.com/github/Ranjankraj/Shala/blob/main/multiclass_20_ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Classification of 20_newsgroup consisting of around 20000 files with 20 classes  
# Random Forrest, Naive bayes and LSTM

#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip  "/content/20_newsgroups.zip"

In [ ]:
#After manually uploading zip file
!unzip -u "/content/20_newsgroups.zip" -d "/content/Internship/"

In [64]:
!ls

20_newsgroups.zip  Internship  sample_data


In [ ]:
from os import listdir
from os.path import isfile, join
# import string

In [ ]:
my_path = "/content/Internship/20_newsgroups"

#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]

In [65]:
folders

['comp.os.ms-windows.misc',
 'sci.med',
 'talk.religion.misc',
 'rec.sport.hockey',
 'talk.politics.guns',
 'comp.graphics',
 'rec.autos',
 'soc.religion.christian',
 'talk.politics.misc',
 'talk.politics.mideast',
 'alt.atheism',
 'sci.electronics',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.motorcycles',
 'misc.forsale',
 'comp.sys.ibm.pc.hardware',
 'sci.space',
 'sci.crypt']

In [ ]:
print(len(folders))

20


In [ ]:
#function to remove incompatible files
import os
import sklearn.datasets
def find_incompatible_files(path):
    """
    Finds the filenames that are incompatible with `CountVectorizer`. These files are usually not compatible with UTF8!
    parameter `path` is the absolute or relative path of the training data's root directory.
    """

    count_vector = sklearn.feature_extraction.text.CountVectorizer()
    files = sklearn.datasets.load_files(path)
    num = []
    for i in range(len(files.filenames)):
        try:
            count_vector.fit_transform(files.data[i:i + 1])
        except UnicodeDecodeError:
            num.append(files.filenames[i])
            os.remove(files.filenames[i])
        except ValueError:
            pass

    return num


In [ ]:
#Removing incompatible files
val = find_incompatible_files(my_path)

In [ ]:
#printing no. of incompatible files removed

print(len(val))

73


In [ ]:
#creating a 2D list to store list of all files in different folders

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [ ]:
#There is around 20000 files or data points to read from
sum(len(files[i]) for i in range(20))

19924

In [ ]:
pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))

In [66]:
len(pathname_list)

19924

In [ ]:
#making an array containing the classes each of the documents belong to

Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [67]:
len(Y)

19924

In [ ]:
# function for reading data

def ps(doc):
      f = open(doc)
      data= f.read()
      return str(data)

In [ ]:
#Reading & storing data from all the files 

import glob
import os.path
import string 

X_data = []

for i in range(len(pathname_list)):
  for file in glob.glob(pathname_list[i]):
      
      data = ps(file)
      X_data.append(data)

In [ ]:
print(len(X_data))

19924


In [68]:
#Sample of texts from one file 

print(X_data[0])

Xref: cantaloupe.srv.cs.cmu.edu comp.os.ms-windows.misc:9169 comp.os.ms-windows.apps:10962
Newsgroups: comp.os.ms-windows.misc,comp.os.ms-windows.apps
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!howland.reston.ans.net!bogus.sura.net!darwin.sura.net!news.udel.edu!bach.udel.edu!swyatt
From: swyatt@bach.udel.edu (Stephen L Wyatt)
Subject: Re: WinBench
Message-ID: <C52pKx.JKE@news.udel.edu>
Sender: usenet@news.udel.edu
Nntp-Posting-Host: bach.udel.edu
Organization: University of Delaware
References: <1pq37s$9dq@seven-up.East.Sun.COM> <1pqp3rINNg85@hp-col.col.hp.com> <rick.734113494@sundance>
Date: Tue, 6 Apr 1993 17:50:57 GMT
Lines: 12

Ok, so if everyone is cheating.. is there any tests that run some 
macro (to load a huge drawing, etc...) on the SAME machine that might 
then tell us what the REAL world results are?

I mean, run the tests on the same machine with different video cards
running word, excel, or something like that to 

In [ ]:
#Organising data in pandas dataframe to work with

import pandas as pd

df1 = pd.DataFrame(X_data, columns=['Text'])
df2 = pd.DataFrame(Y, columns=['Labels'])

df = pd.concat([df1, df2], axis=1)
print(len(df))
df.head()

19924


,Text,Labels
0,Xref: cantaloupe.srv.cs.cmu.edu comp.os.ms-win...,comp.os.ms-windows.misc
1,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,comp.os.ms-windows.misc
2,Newsgroups: comp.os.ms-windows.misc\nPath: can...,comp.os.ms-windows.misc
3,Path: cantaloupe.srv.cs.cmu.edu!rochester!corn...,comp.os.ms-windows.misc
4,Newsgroups: comp.os.ms-windows.misc\nPath: can...,comp.os.ms-windows.misc


In [ ]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize 
from nltk.stem.porter import PorterStemmer

corpus = []

for i in range(len(X_data)):
    
    review = re.sub('[^a-zA-Z]', ' ', df['Text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english')) and len(word)>3]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.Text = corpus

In [69]:
df.Labels = df.Labels.astype('category')
df['Code'] = df.Labels.cat.codes
df.head()

,Text,Labels,Code
0,xref cantaloup comp window misc comp window ap...,comp.os.ms-windows.misc,2
1,path cantaloup magnesium club news ohio state ...,comp.os.ms-windows.misc,2
2,newsgroup comp window misc path cantaloup news...,comp.os.ms-windows.misc,2
3,path cantaloup rochest cornel batcomput munnar...,comp.os.ms-windows.misc,2
4,newsgroup comp window misc path cantaloup news...,comp.os.ms-windows.misc,2


In [ ]:
#List of all words after cleaning

list_of_words = []

for i in range(len(df['Text'])):
  for word in df['Text'][i].split():
    list_of_words.append(word)

In [ ]:
#finding the number of unique words that we have extracted from the documents
import numpy as np

np_list_of_words = np.asarray(list_of_words)
words, counts = np.unique(np_list_of_words, return_counts=True)
len(words)

83755

In [ ]:
from sklearn.model_selection import train_test_split

X = df.Text
y = df.Code
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state = 123, shuffle=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
#defining accuracy function for evaluation of models

def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    get_fit = pipeline.fit(X_train, y_train)
    y_pred1 = get_fit.predict(X_test)
    accuracy1 = accuracy_score(y_test, y_pred1)
    y_pred2 = get_fit.predict(X_train)
    accuracy2 = accuracy_score(y_train, y_pred2)
    print("accuracy score for test: {0:.2f}%".format(accuracy1*100))
    # print(classification_report(y_test, y_pred1))
    print("accuracy score for train: {0:.2f}%\n".format(accuracy2*100))
    # print(classification_report(y_train, y_pred2))
    return accuracy1

In [ ]:
#Constructing a pipeline for model evaluation

import numpy as np
cv = CountVectorizer(max_df=0.5, min_df=100)
rf = RandomForestClassifier()
n_features = np.arange(20000,25001,5000)

def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
        #result contains test accuracy
    return result

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
print("Result for trigram by random forrest (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram by random forrest (Tfidf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


Test result for 20000 features
accuracy score for test: 93.37%
accuracy score for train: 98.04%

Test result for 25000 features
accuracy score for test: 93.46%
accuracy score for train: 98.04%



In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [ ]:
print("Result for trigram by naive bayes (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3), classifier=clf)

Result for trigram by naive bayes (Tfidf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


Test result for 20000 features
accuracy score for test: 90.85%
accuracy score for train: 94.65%

Test result for 25000 features
accuracy score for test: 90.91%
accuracy score for train: 94.66%



In [71]:
#Building multiclass classification LSTM model

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [87]:
n_most_common_words = 25000
max_len=500
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df['Text'].values)
sequences = tokenizer.texts_to_sequences(df['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique words.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 83755 unique words.


In [73]:
Y = pd.get_dummies(df['Code']).values

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=123, shuffle=True)

In [70]:
epochs = 20
emb_dim = 128
batch_size = 500


In [89]:
model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
# model.add(SpatialDropout1D(0.4))
model.add((LSTM(128, dropout=0.4, recurrent_dropout=0.4)))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
clsi = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[EarlyStopping(monitor="val_loss", mode="auto")])


Epoch 1/20
24/24 [==============================] - 55s 2s/step - loss: 2.9889 - acc: 0.0984 - val_loss: 2.9015 - val_acc: 0.2482
Epoch 2/20
24/24 [==============================] - 52s 2s/step - loss: 2.8634 - acc: 0.2106 - val_loss: 2.6323 - val_acc: 0.1900
Epoch 3/20
24/24 [==============================] - 51s 2s/step - loss: 2.5086 - acc: 0.2392 - val_loss: 2.3063 - val_acc: 0.2074
Epoch 4/20
24/24 [==============================] - 52s 2s/step - loss: 2.2593 - acc: 0.3514 - val_loss: 2.0232 - val_acc: 0.3556
Epoch 5/20
24/24 [==============================] - 52s 2s/step - loss: 2.0136 - acc: 0.4126 - val_loss: 1.9138 - val_acc: 0.4125
Epoch 6/20
24/24 [==============================] - 51s 2s/step - loss: 1.7135 - acc: 0.5220 - val_loss: 1.8960 - val_acc: 0.4185
Epoch 7/20
24/24 [==============================] - 53s 2s/step - loss: 1.7189 - acc: 0.4965 - val_loss: 1.5313 - val_acc: 0.5162
Epoch 8/20
24/24 [==============================] - 54s 2s/step - loss: 1.2117 - acc: 0.67

In [90]:
accr1 = model.evaluate(X_test, y_test)
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr1[0], accr1[1]))

156/156 [==============================] - 23s 145ms/step - loss: 0.9899 - acc: 0.6507
Test set
 Loss: 0.990
 Accuracy: 0.651


In [91]:
accr2 = model.evaluate(X_train, y_train)
print('Train set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr2[0], accr2[1]))

467/467 [==============================] - 69s 147ms/step - loss: 0.7040 - acc: 0.7657
Train set
 Loss: 0.704
 Accuracy: 0.766


In [ ]:
#mapping word to document frequency for all unique words

dictionary = {}

for word in words:
  n = 0
  for i in range(len(df['Text'])):
    for w in df['Text'][i].split():
      if word==w:
        n += 1
        continue
  dictionary[word] = n

In [ ]:
#Sorting and building vocabulary for data
import operator

sorted_d = sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True)
vocab_list = list(sorted_d)

In [ ]:
#Probability P(word|class) function

def prob(word):
  for i in range(20):
    for j in range(len(df['Text']):
      n = 0
      m = 0
        if df['Code'][j] ==i:
          m += 1
          for w in df['Text'][j]:
            if word==w:
            n += 1
            continue       
    print("Probability P({}|{}) is equal to {0:.2f}".format(word, i, (n/m)))